In [3]:
%reset

# BMI multinomial Example

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler # Normalization
from sklearn.model_selection import train_test_split # train, test 데이터분리
from sklearn.model_selection import KFold # cross validation

# Raw Data Loading
df = pd.read_csv('./data/bmi.csv', skiprows =3)

# 결측치와 이상치 확인 및 처리 (결측치와 이상치는 없기 떄문에 이부분은 넘어가요

# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height', 'weight']], df['label'], test_size=0.3, random_state = 0)
# 7:3 비율로 train과 test 데이터를 분리했어요!!
# x_data_test, t_data_test 두개는 맨 끝에서 우리 모델의 최종 Accuracy를 측정할 때 한번 사용

# Nomalization 진행(Min-Max Scaling)
scaler = MinMaxScaler()
scaler.fit(x_data_train) # 나중에 Scaling을 하기 위한 정보를 scaler에게 세팅

x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train # 에러를 방지하기 위해서 사용하지 않는 변수를 삭제
del x_data_test

################ Tensorflow 구현 ####################################

## 정답에 해당하는 t_data_train을 살펴보니 이게.. Multinomial 이에요
# 당연히 One-Hot Encording을 이용해서 데이터를 변환시켜야 해요!

# 0 => 1 0 0 0
# 1 => 0 1 0 0
# 2 => 0 0 1 0
# 3 => 0 0 0 1

# 형태로 바꿔줘야 해요!
# Numpy를 이용해서 로직처리로 바꾸는 방법이 있고
# Tensorflow를 이용해서 API를 이용해서 바꾸는 방법이 있어요!
# Tensorflow node를 실행하기 위해서 session이 필요 # one_hot 때문에
sess = tf.Session()

# 우리가 사용할 label(t_data)을 one hot encoding 형태로 변환!!
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=3)) # 결과가 몇개인지 
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=3))

del t_data_train   # 이것도 오류를 방지하기 위해서 제거 !
del t_data_test

# 지금까지 위에서 
# x_data_train_norm, t_data_train_onehot 만들었어요!
# training data set을 준비했어요!

# 데이터가 준비되었으니 이제 Tensorflow Graph를 그려보아요

# Placeholder
X = tf.placeholder(shape=[None, 2], dtype=tf.float32)
T = tf.placeholder(shape=[None, 3], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([2,3]), name='weight')
b = tf.Variable(tf.random.normal([3]), name='bias')

# Hypothesis(Model)
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit) # tf.sigmoid()


# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit, labels = T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)


# 반복학습하는 함수
# parameter
num_of_epoch = 1000
batch_size = 100 # 한번에 학습할 x_data와 t_data의 행의 수

def run_train(sess, train_x, train_t):
    print('##### 학습이 시작됩니다!!!#####')
    sess.run(tf.global_variables_initializer())
    total_batch = int(train_x.shape[0] / batch_size) # (14000, 2)
    
    for step in range(num_of_epoch):
        
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train, loss], feed_dict={X:batch_x,T:batch_t})
            
        if step % 100 ==0:
            print('Loss : {}'.format(loss_val))
    print('#### 학습이 종료되요!! ####')

# run_train(sess, x_data_train_norm, t_data_train_onehot)


# session과 초기화
# sess.run(tf.global_variables_initializer())

# # 반복학습
# for step in range(30000): # 전체 training data로 1번 학습 => 1 epoch
#     _, loss_val = sess.run([train, loss], feed_dict={X:x_data_train_norm,T:t_data_train_onehot})
    
#     if step % 3000 == 0:
#         print('Loss : {}'.format(loss_val))

# Accuracy (정확도측정)      #     0    1    2
predict = tf.argmax(H,1)  # [[0.5  0.4  0.1]] 내가 입력으로 넣은 값에 대한 예측
print(predict)
# 1 axis 방향 지칭
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))


# 학습을 진행하고 
run_train(sess, x_data_train_norm, t_data_train_onehot)

# Accuracy 측정 (Training data 로 validation을 수행해보아요!)
result = sess.run(accuracy, feed_dict={X:x_data_train_norm,T:t_data_train_onehot})
print('Training data 로 validation한 정확도 : {}'.format(result))
# 이렇게 하는건 좋지 않아요!
# 이렇게 하는것 보다는 Cross Validation을 하는게 좋아요!!

# Cross Validation 
cv = 5 # [훈련, 검증] => 5 Set가 만들어져요
results = [] # 5 set 에 대한 accuracy를 구해서 ㅣist 안에 차곡
kf = KFold(n_splits=cv, shuffle = True)


for training_idx, validation_idx in kf.split(x_data_train_norm):
    print(training_idx, validation_idx)
    train_x = x_data_train_norm[training_idx] # Fancy indexing
    train_t = t_data_train_onehot[training_idx]
    valid_x = x_data_train_norm[validation_idx]
    valid_t = t_data_train_onehot[validation_idx]
    
    run_train(sess, train_x, train_t)
    results.append(sess.run(accuracy, feed_dict={X:valid_x, T:valid_t}))

print('cross Validation 결과 : {}'.format(results))
print('cross Validation 최종결과 : {}'.format(np.mean(results)))

# # 학습진행
# run_train(sess, x_data_train_norm, t_data_train_onehot)

# # Accuracy 측정
# result = sess.run(accuracy, feed_dict={X:x_data_test_norm,T:t_data_test_onehot})

# print('최종 정확도 : {}'.format(result))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Tensor("ArgMax_3:0", shape=(?,), dtype=int64)
##### 학습이 시작됩니다!!!#####
Loss : 0.9213409423828125
Loss : 0.20559997856616974
Loss : 0.16243956983089447
Loss : 0.14279884099960327
Loss : 0.1310158669948578
Loss : 0.1229526624083519
Loss : 0.11698706448078156
Loss : 0.11233890801668167
Loss : 0.1085810586810112
Loss : 0.10545748472213745
#### 학습이 종료되요!! ####
Training data 로 validation한 정확도 : 0.9827142953872681
[    1     2     4 ... 13996 13997 13998] [    0     3     7 ... 13988 13989 13999]
##### 학습이 시작됩니다!!!#####
Loss : 0.9648787975311279
Loss : 0.20163343846797943
Loss : 0.1549728959798813
Loss : 0.13373473286628723
Loss : 0.12105464190244675
Loss : 0.11243206262588501
Loss : 0.1060955598950386
Loss : 0.10119026899337769
Loss : 0.0972495749592781
Loss : 0.0939943715929985
#### 학습이 종료되요!! ####
[    0     1     3 ... 13996 13998 13999] [    2     5    12 ... 13993 13995 13997]
##### 학습이 시작됩니다!!!#####
Loss : 0.987198233604431

In [180]:
# Prediction!!
height = 187
weight = 78

my_state = [[height, weight]]
my_state_scaled = scaler.transform(my_state)
print(my_state_scaled)

result = sess.run(H, feed_dict={X:my_state_scaled})
print(result)
print(np.argmax(result))

[[0.8375     0.95555556]]
[[5.762022e-04 9.758552e-01 2.356866e-02]]
1


0
